In [ ]:
#com unsloth para facilitar e reduzir o uso de memória/ser mais rápido

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-99xic8ke/unsloth_a2712968d52a4b249ffec8c8c0be465a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-99xic8ke/unsloth_a2712968d52a4b249ffec8c8c0be465a
  Resolved https://github.com/unslothai/unsloth.git to commit 976d11a10d54383aeb7a692c69e01151a20bfd72
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
#Depedências
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
max_seq_length = 2048
dtype = None
load_in_4bit = True # Quantização 4bit para reduzir o uso de memória

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [9]:
#Inferência antes de aplicar o Fine tuning

#Estrutura de dado para passar par ao modelo
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crie uma questão de programação em Python, sobre Classificação de Temperatura e que envolva o tópico de decisão simples.", # instruction
        "", # input
        "", # output - sempre em branco para receber a geração
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Crie uma questão de programação em Python, sobre Classificação de Temperatura e que envolva o tópico de decisão simples.

### Input:


### Response:
def classificaTemp(temperatura):
    if temperatura < 0:
        return 'Frio'
    elif temperatura > 0:
        return 'Quente'
    else:
        return 'Agora está boa'

print(classificaTemp(-10))
print(classificaTemp(0))
print(classificaTemp(25))<|end_of_text|>


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
#Formatação e passagem de dados de treinamento
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

#local que se encontram os arquivos
localFile = "/content/sample_data/questoes.json"
print(localFile)

#carregando conjunto de dados
dataset = load_dataset("json", data_files={"train":localFile}, split = "train")
print(dataset)

#Aplicando a formatação compatível para o Llama 3.1
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

/content/sample_data/questoes.json


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10
})


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 10
})


In [13]:
#Parâmentros de treino
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
#Realizar o Fine tuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.323600
2,1.259600
3,1.300000
4,1.224500
5,1.182600
6,1.066200
7,0.958200
8,0.875700
9,0.790800
10,0.662000


In [19]:
#Inferência após o Fine tuning
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crie uma questão de programação em Python, sobre Classificação de Temperatura e que envolva o tópico de decisão simples.", # instruction
        "", # input
        "", # output - sempre em branco para receber a geração
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer,)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Crie uma questão de programação em Python, sobre Classificação de Temperatura e que envolva o tópico de decisão simples.

### Input:


### Response:
{Título}
Questão 1 - {Tópicos}Decisão simples{\Tópicos}
{\Título}

{Tema}Classificação de temperatura{\Tema}
{Introdução}
Denomina-se temperatura seca a diferença entre a temperatura real e a umidade relativa do ar, considerando-se que a umidade se evapora com uma taxa proporcional à diferença entre a temperatura e a umidade seca. Por exemplo, se a temperatura real é 25 graus Celsius e a umidade relativa é 60\%, então a temperatura seca é 11 graus Celsius (a umidade se evapora com uma taxa de 4\% de umidade por grau Celsius acima da temperatura). O técnico em dedetização decida como dividir sua equipe entre duas regiões, de modo a fazer o trabalho o 